In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import pandas as pd
import shutil
from pathlib import Path
from config.config import OBSERVACIONS_DIR, TRAIN_TEST_INDEX_DIR, TRAIN_TEST_SERIES_DIR

In [18]:
#if folder train_test_series_dir exists, delete it
if TRAIN_TEST_SERIES_DIR.exists():
    shutil.rmtree(TRAIN_TEST_SERIES_DIR)

#Create it again
TRAIN_TEST_SERIES_DIR.mkdir()


#inside it, create folder called train and test
(TRAIN_TEST_SERIES_DIR / 'train').mkdir()
(TRAIN_TEST_SERIES_DIR / 'test').mkdir()

In [19]:
#for each file in TRAIN_TEST_DIR, read csv

for file in TRAIN_TEST_INDEX_DIR.glob('*.csv'):


    #get name of file without extension
    file_name = file.stem

    #split by _ and remove first element
    cal_val, station = file_name.split('_')[1:]

    original_timeseries_df = pd.read_csv(OBSERVACIONS_DIR / f'{station}.csv', parse_dates=True)

    #new column with index + 1
    original_timeseries_df['index'] = original_timeseries_df.index + 1

    indexs_df = pd.read_csv(file)

    #change the only column name to index
    indexs_df.columns = ['index']

    #merge original_timeseries_df with indexs_df
    original_timeseries_df = pd.merge(original_timeseries_df, indexs_df, on='index')[['Date', 'Flow']].set_index('Date')

    train_test = 'train' if cal_val == 'cal' else 'test'

    #save to csv
    original_timeseries_df.to_csv(TRAIN_TEST_SERIES_DIR / train_test / f'{station}.csv')
